In [1]:
import os, torch
import pandas as pd
from dotenv import load_dotenv
from dags.utils import decode_zip, extract_headers, decode_body
from transformers import AutoModel, AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available()) 
# load_dotenv()
# # SCOPES: Gmail read-only
# SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
# token_path = os.getenv("token_path")
# credentials_path = os.getenv("credentials_path")

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)
model_2 = AutoModel.from_pretrained(model_name)

PyTorch version: 2.8.0+cu129
CUDA available: True


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
path_1 = "data/imp_22-08-2025-03-23.json.gz"
path_2 = "data/unimp_22-08-2025-03-23.json.gz"
decompressed_data_1 = decode_zip(path_1)
decompressed_data_2 = decode_zip(path_2)

df_imp =pd.DataFrame(decompressed_data_1)
df_unimp =pd.DataFrame(decompressed_data_2)

df_imp["Subject"] = df_imp["Payload"].apply(lambda x: pd.Series(extract_headers(x)))
df_imp["Body"] = df_imp["Payload"].apply(decode_body)
df_imp["Important"] = 1
df_imp = df_imp.drop(["Id", "Payload"], axis=1)

df_unimp["Subject"] = df_unimp["Payload"].apply(extract_headers)
df_unimp["Body"] = df_unimp["Payload"].apply(decode_body)
df_unimp["Important"] = 0
df_unimp = df_unimp.drop(["Id", "Payload"], axis=1)
df_unimp.head(2)

,Subject,Body,Important
0,artificial intelligence engineer the reliable ...,the reliable jobs artificial intelligence engi...,0
1,get 7 days of unrestricted learning,hey sathwik have you ever felt that little spa...,0


In [18]:
df_imp.head(2)

,Subject,Body,Important
0,important scheduled maintenance affecting serv...,hello sathwik please note there will be essent...,1
1,urgent housing application expiry warning,hi sathwik your housing application will expir...,1


In [19]:
train = pd.concat([df_imp, df_unimp])
train.isnull().sum()

Subject       0
Body         53
Important     0
dtype: int64

In [ ]:
train["Body"] = train["Body"].fillna(train["Subject"])

train["subject_tokenized"] = train["Subject"].apply(lambda x: tokenizer(x, truncation=True, max_length=50, padding=True, return_tensors="pt"))
train["body_tokenized"] = train["Body"].apply(lambda x: tokenizer(x, truncation=True, max_length=512, padding=True, return_tensors="pt"))

In [22]:
train.head(2)

,Subject,Body,Important,subject_tokenized,body_tokenized
0,important scheduled maintenance affecting serv...,hello sathwik please note there will be essent...,1,"[input_ids, attention_mask]","[input_ids, attention_mask]"
1,urgent housing application expiry warning,hi sathwik your housing application will expir...,1,"[input_ids, attention_mask]","[input_ids, attention_mask]"


In [45]:
body_tokens = train.loc[:, "body_tokenized"].values
subject_tokens = train.loc[:, "subject_tokenized"].values
labels = train.loc[:,"Important"].values 

In [40]:
import umap
import numpy as np
import matplotlib.pyplot as plt
from dags.utils import get_embeddings_from_token

In [46]:
body_embd = get_embeddings_from_token(model_name, body_tokens)

In [47]:
subject_embd = get_embeddings_from_token(model_name, subject_tokens)

In [15]:
# # UMAP reduction to 2D
# umap_model = umap.UMAP(n_neighbors=10, min_dist=0.3, random_state=42, n_components=2)
# emb_2d = umap_model.fit_transform(embeddings)

# # Some Overlap is expected
# #High-dimensional separation may not be visible in 2d
# #Can try diff embedding/tokenizer


# # Scatterplot
# plt.figure(figsize=(6, 4))
# scatter = plt.scatter(emb_2d[:, 0], emb_2d[:, 1], c=labels, cmap="Spectral", alpha=0.8)
# plt.xlabel("UMAP-1")
# plt.ylabel("UMAP-2")
# plt.title("Sentence Embeddings with UMAP (colored by label)")
# plt.colorbar(scatter, label="Dependent Variable (0/1)")
# plt.show()

In [ ]:
# X = embd
# y = labels

In [ ]:
# from sklearn.model_selection import train_test_split
# import xgboost as xgb
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=True, random_state=42)


# dtrain = xgb.DMatrix(X_train, label=y_train)
# params = {
#     "objective": "binary:logistic", 
#     "eval_metric": "logloss",
#     "eta": 0.1,
#     "max_depth": 6
# }
# model = xgb.train(params, dtrain, num_boost_round=200)

In [ ]:
# dunlabeled = xgb.DMatrix(X_test)
# y_pred_proba = model.predict(dunlabeled)